In [1]:
# File for classification

In [2]:
from os.path import join
from google.colab import drive

# path to your project on Google Drive
MY_GOOGLE_DRIVE_PATH = 'My Drive/Capstone Project'
ROOT = '/content/gdrive/'
drive.mount(ROOT)

Mounted at /content/gdrive/


In [30]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import random_split
from collections import Counter
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.compose import make_column_transformer
# from sklearn.model_selection import train_test_split


In [13]:
#Setting Random Seed
np.random.seed(0)
torch.manual_seed(0)

In [14]:
#Setting Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [15]:
# import churn and telescope datasets
data_dir = ROOT + MY_GOOGLE_DRIVE_PATH + '/Edge/data/'

churn = pd.read_csv(data_dir + 'churn.csv')
churn.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

telescope = pd.read_table(data_dir + 'telescope.dat', header=None, delimiter=',')
telescope.columns = ['FLength', 'FWidth', 'FSize', 'FConc', 'FConc1', 'FAsym', 
                     'FM3Long', 'FM3Trans', 'FAlpha', 'FDist', 'Class']

print('Shape - Churn: {} & Telescope: {}'.format(churn.shape, telescope.shape))

Shape - Churn: (10000, 11) & Telescope: (19020, 11)


In [16]:
churn.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [17]:
churn["Exited"].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [18]:
churn["Geography"].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [19]:
churn["Gender"].value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

In [20]:
# preprocess churn & telescope data, convert numpy to tensor
preprocess = make_column_transformer(
    (OneHotEncoder(), ['Geography', 'Gender']), remainder = StandardScaler())

In [21]:
data = preprocess.fit_transform(churn.iloc[:, :-1])
target = np.array(churn.iloc[:, -1])
print('\nClass Distribution - Churn:', Counter(target))
data = data.astype(np.float32)
target = target.astype(np.long)

churn = {'data': torch.from_numpy(data), 'target': torch.from_numpy(target)}


Class Distribution - Churn: Counter({0: 7963, 1: 2037})


In [22]:
churn_dataset = torch.utils.data.TensorDataset(churn["data"],churn["target"])

In [23]:
telescope.head()

,FLength,FWidth,FSize,FConc,FConc1,FAsym,FM3Long,FM3Trans,FAlpha,FDist,Class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


In [24]:
telescope["Class"].value_counts()

g    12332
h     6688
Name: Class, dtype: int64

In [25]:
telescope.describe()

,FLength,FWidth,FSize,FConc,FConc1,FAsym,FM3Long,FM3Trans,FAlpha,FDist
count,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000
mean,53.250154,22.180966,2.825017,0.380327,0.214657,-4.331745,10.545545,0.249726,27.645707,193.818026
std,42.364855,18.346056,0.472599,0.182813,0.110511,59.206062,51.000118,20.827439,26.103621,74.731787
min,4.283500,0.000000,1.941300,0.013100,0.000300,-457.916100,-331.780000,-205.894700,0.000000,1.282600
25%,24.336000,11.863800,2.477100,0.235800,0.128475,-20.586550,-12.842775,-10.849375,5.547925,142.492250
50%,37.147700,17.139900,2.739600,0.354150,0.196500,4.013050,15.314100,0.666200,17.679500,191.851450
75%,70.122175,24.739475,3.101600,0.503700,0.285225,24.063700,35.837800,10.946425,45.883550,240.563825
max,334.177000,256.382000,5.323300,0.893000,0.675200,575.240700,238.321000,179.851000,90.000000,495.561000


In [26]:
scaler = StandardScaler()

In [27]:
data = scaler.fit_transform(telescope.iloc[:, :-1])
target = LabelEncoder().fit_transform(telescope.Class)
print('\nClass Distribution - Telescope:', Counter(target))

data = data.astype(np.float32)
target = target.astype(np.long)

telescope = {'data': torch.from_numpy(data), 'target': torch.from_numpy(target)}


Class Distribution - Telescope: Counter({0: 12332, 1: 6688})


In [28]:
telescope_dataset = torch.utils.data.TensorDataset(telescope["data"],telescope["target"])

In [ ]:
#hyperparameters

hidden_size = 5
num_classes = 2
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [ ]:
#Model Architecture
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=churn_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

In [ ]:
input_size = churn["data"].shape[1]
model_churn = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_churn.parameters(), lr=learning_rate)  

In [ ]:
print(model_churn)

NeuralNet(
  (fc1): Linear(in_features=13, out_features=5, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=5, out_features=2, bias=True)
)


In [ ]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        features = features.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model_churn(features)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/5], Step [100/100], Loss: 0.5410
Epoch [2/5], Step [100/100], Loss: 0.4922
Epoch [3/5], Step [100/100], Loss: 0.4504
Epoch [4/5], Step [100/100], Loss: 0.3604
Epoch [5/5], Step [100/100], Loss: 0.4993


In [ ]:
# run a test loop
test_loss = 0
correct = 0
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    data = data.view(-1, 28 * 28)
    net_out = net(data)
    # sum up batch loss
    test_loss += criterion(net_out, target).data[0]
    pred = net_out.data.max(1)[1]  # get the index of the max log-probability
    correct += pred.eq(target.data).sum()

test_loss /= len(test_loader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
l = [module for module in model_churn.children() if type(module) != nn.Sequential]
print(l)
for i in l:
  try:
    print('Layer: {} weight:{}'.format(i,i.weight))
  except:
    continue

[Linear(in_features=13, out_features=5, bias=True), ReLU(), Linear(in_features=5, out_features=2, bias=True)]
Layer: Linear(in_features=13, out_features=5, bias=True) weight:Parameter containing:
tensor([[-0.0864,  0.0676, -0.2783, -0.2585, -0.2018,  0.0066,  0.0375,  0.1235,
          0.0326,  0.0747, -0.0436, -0.0262, -0.1691],
        [-0.2319, -0.1648, -0.0956,  0.0858,  0.0710, -0.1671,  0.0069,  0.0429,
          0.1638,  0.1675,  0.1183, -0.0709,  0.0955],
        [ 0.4540, -0.1149,  0.1110,  0.0815,  0.1665,  0.3673, -0.2699, -0.1216,
         -0.3564, -0.0590, -0.0800,  0.3081, -0.0461],
        [ 0.3311,  0.0950,  0.1657,  0.1592,  0.0199, -0.1524, -0.3282,  0.3414,
          0.2048, -0.1104,  0.1747,  0.2841,  0.1523],
        [ 0.2829,  0.1286,  0.4235, -0.0565,  0.2739, -0.0910, -0.4560, -0.0566,
         -0.0262,  0.1511, -0.0297,  0.0409, -0.0612]], device='cuda:0',
       requires_grad=True)
Layer: Linear(in_features=5, out_features=2, bias=True) weight:Parameter contai

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=telescope_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

In [ ]:
input_size = telescope["data"].shape[1]
model_telescope = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_telescope.parameters(), lr=learning_rate)  

In [ ]:
print(model_telescope)

NeuralNet(
  (fc1): Linear(in_features=10, out_features=5, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=5, out_features=2, bias=True)
)


In [ ]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        features = features.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model_telescope(features)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/5], Step [100/191], Loss: 0.6065
Epoch [2/5], Step [100/191], Loss: 0.4885
Epoch [3/5], Step [100/191], Loss: 0.3402
Epoch [4/5], Step [100/191], Loss: 0.4141
Epoch [5/5], Step [100/191], Loss: 0.4768


In [ ]:
l = [module for module in model_telescope.children() if type(module) != nn.Sequential]
print(l)
for i in l:
  try:
    print('Layer: {} weight:{}'.format(i,i.weight))
  except:
    continue

[Linear(in_features=10, out_features=5, bias=True), ReLU(), Linear(in_features=5, out_features=2, bias=True)]
Layer: Linear(in_features=10, out_features=5, bias=True) weight:Parameter containing:
tensor([[-0.5308, -0.0077,  0.4260, -0.5823, -0.4349, -0.0803,  0.2500, -0.0241,
         -0.5900,  0.0134],
        [-0.1393, -0.2644, -0.1518,  0.0024, -0.4289,  0.0300,  0.1299,  0.0241,
         -0.9110,  0.2471],
        [ 0.7236,  0.1661, -0.0161, -0.3977, -0.0416, -0.0188, -0.1537,  0.0290,
          0.5282, -0.0181],
        [-0.1120, -0.4191,  0.3245,  0.1274,  0.1254,  0.4344,  0.5038,  0.0107,
          0.0055,  0.0985],
        [ 0.1704, -0.0345,  0.3293,  0.0144, -0.0237, -0.2759, -0.0182, -0.0754,
          0.4712,  0.4365]], device='cuda:0', requires_grad=True)
Layer: Linear(in_features=5, out_features=2, bias=True) weight:Parameter containing:
tensor([[ 0.1503,  0.5413, -0.6207, -0.2329, -0.2639],
        [-0.7058, -0.4487,  0.3611,  0.6021,  0.6992]], device='cuda:0',
       r